In [1]:
import torch

In [30]:
#设置 requires_grad=True 来跟踪与它相关的计算,因此x.grad是另一个张量，其保持x相对于某个标量值的梯度。
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [31]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [32]:
z=y*y
print(z,z.mean())

tensor([[9., 9.],
        [9., 9.]], grad_fn=<MulBackward0>) tensor(9., grad_fn=<MeanBackward0>)


In [33]:
out = z.mean()
print(out)

tensor(9., grad_fn=<MeanBackward0>)


In [34]:
out.backward()
print(out)

tensor(9., grad_fn=<MeanBackward0>)


In [14]:
# d(out)/dx
x.grad

tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])

In [2]:
dtype = torch.float
#device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [3]:
# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [4]:
learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; 
    # these are exactly the same operations we used to compute the forward pass using Tensors, 
    # but we do not need to keep references to intermediate values since we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()
    
    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    # 手动更新weight，或者使用optimizer来更新
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()   

99 746.6549072265625
199 3.7035281658172607
299 0.024944860488176346
399 0.0004014459263999015
499 5.344969758880325e-05
